In [1]:
import pandas as pd
import os
import requests
import numpy as np
import json

In [2]:
import tweepy

# Read in the config file with your personal keys, secrets, and tokens
with open('config_file.txt') as file:
    lines = file.readlines()
    consumer_key = lines[0]
    consumer_secret = lines[1]
    access_token = lines[2]
    access_secret = lines[3]
    
# Set up the API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)